In [55]:
#to perform partial correlation analysis we need to derive raster of valid lsp change
# for that we create a new raster and apply lsp_change information from ecoregion analysis over each SU

import numpy as np  
import pandas as pd
import rasterio as rio  
import xarray as xr
import rioxarray as rxr 
import os   
import geopandas as gpd

In [56]:
#load trend and dem rasters
elev_rxr = rxr.open_rasterio(r"..\Data\DEM_Rasters\elevation.tif")
aspect_rxr = rxr.open_rasterio(r"..\Data\DEM_Rasters\aspect.tif")
slope_rxr = rxr.open_rasterio(r"..\Data\DEM_Rasters\slope.tif")
ecoregion_rxr = rxr.open_rasterio(r"..\Data\Ecoregion_raster\ecoregions_raster.tif")

roi_gdf = gpd.read_file(r"../Data/roi_nepal/nepal_actual_roi.shp")
roi = roi_gdf.to_crs("EPSG:4326")
lsp_metrics = ['sos', 'eos', 'los', 'pos']

base_outdir = r"../Data/Processed/"

In [57]:
# Elevation classes: <1000: 1, 1000-2000: 2, 2000-3000: 3, 3000-4000: 4, >4000: 5
elev_class = xr.where(elev_rxr < 1000, 1, 
                      xr.where(elev_rxr < 2000, 2,
                               xr.where(elev_rxr < 3000, 3,
                                        xr.where(elev_rxr < 4000, 4, 5))))

# Slope classes: 0-2: 1, 2-15: 2, 15-30: 3, >30: 4              Reference: FAO(2006)
slope_class = xr.where(slope_rxr < 2, 1,
                       xr.where(slope_rxr < 15, 2,
                                xr.where(slope_rxr < 30, 3, 4)))

# Aspect classes: Northern (270-360, 0-90): 1, Southern (90-270): 2
aspect_class = xr.where((aspect_rxr >= 0) & (aspect_rxr <= 90), 1,
                        xr.where((aspect_rxr >= 270) & (aspect_rxr <= 360), 1,
                                 xr.where((aspect_rxr > 90) & (aspect_rxr < 270), 2, np.nan)))

for metric in lsp_metrics:
    sig_trend_rxr = rxr.open_rasterio(r"..\Data\Trend_Rasters\mod_"+metric+"_mk_significant.tif")
    aligned_rasters = []
    ref = sig_trend_rxr.rio.clip(roi.geometry, roi.crs, drop=True)

    raster_dict = {
            "ecoregion": ecoregion_rxr,
            "elevation": elev_class,
            "slope": slope_class,
            "aspect": aspect_class,
            "sig_trend": sig_trend_rxr
        }

    for name, raster in raster_dict.items():
        raster = raster.rio.write_crs("EPSG: 4326")
        reproj = raster.rio.reproject_match(ref)
        reproj.name = name
        reproj = reproj.squeeze('band', drop = True)
        aligned_rasters.append(reproj)

    stacked_xr = xr.merge(aligned_rasters)
    stacked_df = stacked_xr.to_dataframe().reset_index()
    # Map lsp_change from spatial unit analysis to each pixel
    # Load the spatial unit trend rules
    su_trend_df = pd.read_csv(r"..\Data\Processed\Ecoregion_trends\sos\All_SU_Stats_sos.csv")

    su_lookup = su_trend_df[['ecoregion', 'elevation', 'slope', 'aspect', 'lsp_change']]

    stacked_df1 = stacked_df.merge(
    su_lookup,
    on=['ecoregion', 'elevation', 'slope', 'aspect'],
    how='left' 
    )

    select_ecoregions = [81003, 40115, 40301, 40403, 40401, 40166, 81021, 40701, 40501, 40502, 40120]

    #apply 0 to all nans, all sig_trend = -999, all ecrs outside select ecrs
    stacked_df1['lsp_change'] = stacked_df1['lsp_change'].fillna(0)
    stacked_df1.loc[stacked_df1['sig_trend'] == -999, 'lsp_change'] = 0
    stacked_df1.loc[~stacked_df1['ecoregion'].isin(select_ecoregions), 'lsp_change'] = 0
    lsp_change_df = stacked_df1[['x', 'y', 'lsp_change']]
    lsp_change_xr = lsp_change_df.set_index(['y', 'x'])['lsp_change'].to_xarray()
    lsp_change_xr = lsp_change_xr.reindex_like(ref)
    lsp_change_xr.rio.write_crs(ref.rio.crs, inplace=True)
    lsp_change_xr.rio.to_raster(r"..\Data\Processed\Valid_lsp_change_"+metric+".tif", driver='GTiff', compress='LZW')

C:\Users\A S U S\AppData\Local\Temp\ipykernel_17368\3034802979.py:37: FutureWarning: In a future version of xarray the default value for compat will change from compat='no_conflicts' to compat='override'. This is likely to lead to different results when combining overlapping variables with the same name. To opt in to new defaults and get rid of these warnings now use `set_options(use_new_combine_kwarg_defaults=True) or set compat explicitly.
  stacked_xr = xr.merge(aligned_rasters)
C:\Users\A S U S\AppData\Local\Temp\ipykernel_17368\3034802979.py:37: FutureWarning: In a future version of xarray the default value for compat will change from compat='no_conflicts' to compat='override'. This is likely to lead to different results when combining overlapping variables with the same name. To opt in to new defaults and get rid of these warnings now use `set_options(use_new_combine_kwarg_defaults=True) or set compat explicitly.
  stacked_xr = xr.merge(aligned_rasters)
C:\Users\A S U S\AppData\L